In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip uninstall keras-nightly

In [ ]:
 #!pip uninstall -y tensorflow

In [ ]:
 !pip install h5py==2.10.0

In [ ]:
!pip install tensorflow-gpu==1.15

In [ ]:
!pip install keras==2.2.4

In [ ]:
!ls "/content/gdrive/My Drive/Mask_RCNN/"

assets				  mask_rcnn.egg-info
build				  mask_rcnn_PP_info_header_best.h5
dist				  mrcnn
images				  PPI
indextable			  PPIExtStarter.h5
last_mask_rcnn_model_py_3_layers  pp_info_legend_cfg20210601T1017
LICENSE				  pp_info_legend_cfg20210601T1019
MANIFEST.in			  README.md
Map_Grid_info			  requirements.txt
MapGridInfoLegendTables		  samples
Map_Grid_info_no_row_col	  setup.cfg
mask_rcnn_coco.h5		  setup.py


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Mask_RCNN/')

In [ ]:
%cd "/content/gdrive/My Drive/Mask_RCNN/"

/content/gdrive/My Drive/Mask_RCNN


In [ ]:
#!pip install -r requirements.txt

In [ ]:
!python3 setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:672: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:672: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:672: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
adding license file 'LICENSE' (matched pattern 'LICEN

In [ ]:
# plot one photograph and mask
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils1 import Dataset
#from matplotlib import pyplot
#from mrcnn.visualize import display_instances
#from mrcnn.utils import extract_bboxes
from mrcnn.config1 import Config
from mrcnn.model1 import MaskRCNN
import imgaug

Using TensorFlow backend.


In [ ]:
######################### Prepare Dataset ##################################
category = 'dataset'
class_names = ["Grid_info","legend"]

# class that defines and loads the kangaroo dataset
class PPInfoDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        #self.add_class("dataset", 1, "table")
        for i in range(1,len(class_names)+1):
            self.add_class(category, i, class_names[i-1])
        # define data locations
        if is_train:
            images_dir=dataset_dir+'/train/'
            annotations_dir=dataset_dir+'/annots/'
        else:
            images_dir=dataset_dir+'/val/'
            annotations_dir=dataset_dir+'/annots/'
        # find all images
        c=0
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename[:-4] # filename is for instance 0001.jpg so
            print("image_id : ",image_id)
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            c=c+1
            # add to dataset
            self.add_image(category, image_id=image_id, path=img_path, annotation=ann_path)
    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        box_class_name = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        for name in root.findall('.//name'):
            box_class_name.append(name.text)
        # extract image dimensions

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height,box_class_name

    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location , xml file
        path = info['annotation']
        # load XML
        boxes, w, h,box_class_name = self.extract_boxes(path)

        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            #row_s , col_s is the starting position (i,j) of box
            #row_e , col_e is the ending position (i,j) of box
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(box_class_name[i]))


        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
################################ Start Training ##############################

#set paths
dataset_path ='/content/gdrive/My Drive/Mask_RCNN/MapGridInfoLegendTables'
model_directory_path='/content/gdrive/My Drive/Mask_RCNN'
pretrained_weights_path='/content/gdrive/My Drive/Mask_RCNN/mask_rcnn_coco.h5'
class PPInfoConfig(Config):
    # Give the configuration a recognizable name
    NAME = "PP_info_Legend_cfg"
    # Number of classes (background + kangaroo)
    NUM_CLASSES = 1 + len(class_names)
    STEPS_PER_EPOCH = 288
    VALIDATION_STEPS = 33
    BACKBONE = "resnet101"
    IMAGES_PER_GPU = 2
    TRAIN_BN = True









# prepare train set
print("############################# TRAIN #########################")
train_set = PPInfoDataset()
train_set.load_dataset(dataset_path , is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
print("############################# TEST #########################")
test_set = PPInfoDataset()
test_set.load_dataset(dataset_path , is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# prepare config
config = PPInfoConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir=model_directory_path , config=config)
# load weights (mscoco) and exclude the output layers   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"]
model.load_weights(pretrained_weights_path, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
#model.load_weights(pretrained_weights_path, by_name=True)
# train weights (output layers or 'heads')
aug = imgaug.augmenters.Sometimes(0.5,imgaug.augmenters.OneOf(
                                            [
                                            imgaug.augmenters.GaussianBlur(sigma=(0, 6)),
                                            imgaug.augmenters.Affine(rotate=(-90, 90)),
                                            imgaug.augmenters.Affine(scale=(0.5, 1.5))
                                             ]
                                        )
                                   )
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all',augmentation = aug)

 #model.keras_model.summary()
# imgaug.Sometimes(5/6,aug.OneOf(
#                                             [
#                                             imgaug.augmenters.Fliplr(1),
#                                             imgaug.augmenters.Flipud(1),
#                                             imgaug.augmenters.Affine(rotate=(-45, 45)),
#                                             imgaug.augmenters.Affine(rotate=(-90, 90)),
#                                             imgaug.augmenters.Affine(scale=(0.5, 1.5))
#                                              ]
#                                         )
#                                   )


############################# TRAIN #########################
image_id :  massavi_27
image_id :  massavi_13
image_id :  massavi_1
image_id :  massavi_32
image_id :  massavi_33
image_id :  massavi_42
image_id :  massavi_47
image_id :  massavi_71
image_id :  massavi_58
image_id :  massavi_79
image_id :  massavi_83
image_id :  massavi_78
image_id :  massavi_97
image_id :  massavi_113
image_id :  massavi_110
image_id :  massavi_111
image_id :  massavi_144
image_id :  massavi_174
image_id :  massavi_160
image_id :  massavi_166
image_id :  massavi_197
image_id :  massavi_211
image_id :  massavi_244
image_id :  massavi_227
image_id :  massavi_269
image_id :  massavi_256
image_id :  massavi_305
image_id :  massavi_302
image_id :  massavi_346
image_id :  massavi_318
image_id :  massavi_323
image_id :  massavi_405
image_id :  massavi_431
image_id :  massavi_437
image_id :  massavi_463
image_id :  massavi_448
image_id :  massavi_467
image_id :  massavi_481
image_id :  massavi_489
image_id :  mass

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/100


ResourceExhaustedError: ignored